http://judge.mipt.ru/mipt_cs_on_python3/labs/lab14.html \
Используется A_stack из лекции

In [2]:
import sys
sys.path.append('../lib')
import A_stack

#### Стеки
В этой лабораторной работе многократно будет использоваться стек,
поэтому для начала предлагается освежить в памяти,
как эта структура обычно выглядит на языке python. \
#### Упражнение №1 
Напишите программу, которая последовательно кладет в стек произвольный массив чисел, а затем распечатывает их , последовательно доставая, пока стек не пуст (например, набор (1,...,10)).

In [3]:
R=range(1,11)
A=A_stack
A.clear()
for x in R:
    A.push(x)
while not A.is_empty():
    print(A.pop())

10
9
8
7
6
5
4
3
2
1


#### Правильные скобочные структуры
Известно, что скобочные выражения могут быть корректными и некорректными. Корректные - это такие, которые можно вычислить по правилам, если расставить между ними числа и знаки бинарных операций. Иными словами, интуитивно, такие. где зоны действия скобок разных типов не пересекаются(если есть скобки разных типов).

#### Упражнение №2
Написать программу, определяющую правильность введенного скобочного выражения, в котором используются скобки 3 видов: (), {}, /. - не очень поняла, что за скобка / пусть будет []


In [4]:
def is_braces_sequense_correct(s:str,braces=[("(",")"),("[","]"),('{','}')]):
    """ Определяет является ли последовательность скобок в строке верной
    Принимает собственно строку и список скобок,
    каждый эл-т списка - кортеж - открывающаяся и закрывающаяся скобка
    >>> is_braces_sequense_correct("(([])){}")
    True
    >>> is_braces_sequense_correct("(([]){)}")
    False
    >>> is_braces_sequense_correct("(([]){)}",braces=[("(",")"),("[","]")])
    True
    >>> is_braces_sequense_correct("{{()")
    False
    >>> is_braces_sequense_correct("]")
    False
    >>> is_braces_sequense_correct("[({})]")
    True
    """
    A=A_stack
    A.clear()
    braces_str=""
    # наши скобки в одной строке
    for l,r in braces:
        braces_str+=l+r
    for brace in s:
        if not brace in braces_str:
            continue
        for left,right in braces:
            if brace==left:
                A.push(brace)
                break
            elif brace==right:
                if A.is_empty() or A.pop()!=left:
                    return False
    return A.is_empty()
    
if __name__== "__main__":
    import doctest
    doctest.testmod()

#### Арифметические выражения. Инфиксная, постфиксная и префиксная нотации
Рассмотрим арифметическое выражение
\begin{equation*} (2-3)*(12-10)+4/2 \end{equation*}
Его значение легко вычисляется и оказывается даже целым - это 0(не забываем правила приоритета операций!). Это привычная для нас форма записи арифметических выражений, в которой если операция бинарная (т.е. требует 2 аргументов, например, сложение, деление), то один аргумент пишется перед знаком операции, а другой - после неё. Такая форма записи называется инфиксной.

Нотация (способ записи), в которой операнды пишутся перед знаком операции - называется постфиксной или обратной польской.

Нотация. в которой операнды пишутся после знака операции - прямой польской или префиксной.

Например, уже рассмотренное выражение в обратной польской записи будет выглядеть как

\begin{equation*} 2 3 - 12 10 - * 4 2 / + \end{equation*}
После пары 2 и 3 стоит знак вычитания. После пары 12 и 10 - тоже. Далее стоит знак перемножения, потому что результаты этих двух операций надо умножить. Далее стоят 4 и 2 и после них знак деления. А после - знак сложения, показывающий, что результат предыдущего нужно сложить с результатом деления 4 на 2.

а в прямой польской как

\begin{equation*} + * - 2 3 - 12 10 / 4 2 \end{equation*}
Аналогичным образом, только теперь знак операции стоит перед операндами (или их описаниями в виде выражений в той же форме записи).

#### Упражнение №3
Перевести выражение

\begin{equation*} (3+4*(2-1))/5 \end{equation*}
в прямую и обратную польские записи.

В общем и целом, любое выражение можно представить в виде структуры, называемой деревом(синтаксическим деревом в данном случае, поскольку оно отражает структуру выражения). Например, для разобранного выражения синтаксическое дерево будет выглядеть так:

![](stree.png)
Его конечные вершины, листья (из которых стрелки никуда не идут) - это операнды, а промежуточные (из которых идут стрелки)- операции. Прямая польская запись (префиксная) получается, если читать это дерево сверху вниз. Обратная (постфиксная) - если читать снизу вверх.

Здесь пока не надо писать перевод всех выражений - это будет позже
\begin{equation*} (3+4*(2-1))/5 \end{equation*}
ППН : $/+3*4-215 $ \
ОПН: $ 3421-*+5/$

#### Стековый калькулятор
Стековый калькулятор - это устройство (реальное или виртуальное), которрое вычисляет значения арифметических выражений , записанный в постфиксной форме. Стековым он называется по понятным причинам: если нам встречается число - мы должны поместить его в стек; если знак опренрации - мы должны достать из стека столько чиссел, скольок необходимо для выполнения данной операции. Именно так и работает т.н. математический сопроцессор (например, в INtel8086+AMD64)- часть ЦП, отвечающая за операции над числами с плавающей запятой.

##### Упражнение №4
Реализовать стековый калькулятор на python. Написать программу, которая читает выражение в обратной польской нотации и считает его значение или пишет, что выражение составлено не корректно (если оно некорректно).

In [5]:
def stack_calculator(equation:list):
    """ Читает выражение в обратной польской нотации
    считает его значение или пишет что оно составлено некорректно
    допустимые операции + - * / ^ 
    >>> stack_calculator([3,4,2,1,"-","*","+",5,"/"])
    1.4
    >>> stack_calculator([2,3,"-",12,10,"-","*",4,2,"/","+"])
    0.0
    >>> stack_calculator([2,3,"^"])
    8
    >>> stack_calculator([2,"+",3])
    Некорректное выражение
    False
    >>> stack_calculator([3,2,2,"-","/"])
    Деление на 0 в выражении
    False
    >>> stack_calculator([-1,1.33,"^"])
    Комплексные числа в вычислении выражения
    False
    >>> stack_calculator([2,3,"*",9])
    Некорректное выражение
    False
    >>> stack_calculator([])
    Некорректное выражение
    False
    """
    """>>> stack_calculator(['1','2',"+"])
    Некорректные символы в выражении
    False
    """
    operations="+-*/^"
    A=A_stack
    A.clear()
    for symbol in equation:
        if type(symbol)==int or type(symbol)==float:
            A.push(symbol)
            continue
        if type(symbol)!=str:
            print("Некорректые символы в выражении")
            return False
        elif symbol in operations:
            if A.size()<2:
                print("Некорректное выражение")
                return False
            y=A.pop()
            x=A.pop()
            if symbol=='+':
                res=x+y
            elif symbol=='-':
                res=x-y
            elif symbol=='*':
                res=x*y
            elif symbol=='/':
                try:
                    res=x/y
                except ZeroDivisionError:
                    print("Деление на 0 в выражении")
                    return False
            elif symbol=='^':
                res=x**y
                if type(res)==complex:
                    print("Комплексные числа в вычислении выражения")
                    return False
            A.push(res)
        #else:
         #   print("Некорректные символы в выражении")
          #  return False
    if not A.size()==1:
        print("Некорректное выражение")
        return False
    return A.pop()

if __name__== "__main__":
    import doctest
    doctest.testmod()


#### Сортировочная станция Дейкстры
Как нетрудно видеть: выражения в обратной польской записи удобны для чтения компьютером , но неудобны для чтения и составления людьми. Поэтому хотелось бы доверить труд составления постфиксной формы выражений по их привычной инфиксной форме компьютеру. Это можно сделать с помощью так называемого алгоритма сортировочной станции (Shunting Yard algorithm , придуман Э. Дейкстрой в 1961 году, см https://en.wikipedia.org/wiki/Shunting-yard_algorithm для трансляторов языка Algol60 https://ir.cwi.nl/pub/9251).

В следующем примере, взятом как раз из википедии, разбирается преобразование выражения

a+b∗c−d \
Есть сортировочная станция с 3 путями: 2 подъездных и 1 тупик. С правоого подъездного пути едут выражения: в каждом "вагоне" или операнд или знак операции. Операнды свободно проезжают в левый путь (образуя очередь), а операторы заезжают в тупик.


By Salix alba - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=10960619

Если приоритет входящего оператора ниже приоритета того, что навершине стека (на входе в тупик), то из стека достаётся оператор с большим приоритетом (на шаге g это умножение). То же самое происходит, если приооритет приходящего оператора равен приоритету такового на вершине стека, но тот, что на вершине - ассоциативен. В данном случае после отъезда умножения остаётся сложение, приоритет которого равен приоритету вычитания, но сложение в отличие от вычитания ассоциативно:

(a+b)+с=a+(b+c)(a−b)−c!=a−(b−c)=a−b+c

#### Упражнение №5
Реализовать на python алгоритм сортировочной станции для преобразования произвольных арифметических выражений с 4 действиями (+,-,*,/) из инфиксной записи в обратную польскую.

In [6]:
def parse_equation_from_infix_to_rpn_no_braces(s:str):
    """Dijkstra shunting yard algorithm
    Алгоритм сортировочной станции Дейкстры
    Преобразует математическое выражение из инфиксной(обычной) нотации в
    обратную польскую нотацию
    принимает выражение в виде списка операндов и операций
    возвращает список - выражение в опн 
    Допустимые арифметические операции + - * / ^
    >>> parse_equation_from_infix_to_rpn_no_braces([])
    []
    >>> #[3,4,2,1,"-","*","+",5,"/"]
    >>> parse_equation_from_infix_to_rpn_no_braces([])
    []
    >>> # [2,3,"-",12,10,"-","*",4,2,"/","+"]
    >>> parse_equation_from_infix_to_rpn_no_braces([1,"+",2,"/",4,"-",3,'*',2,"^",-1])
    [1, 2, 4, '/', '+', 3, 2, -1, '^', '*', '-']
    """
    operators="-+/*^"
    A=A_stack
    A.clear()
    rpn=[]
    for oper in s:
        if type(oper)!=str or not oper in operators:
            rpn.append(oper)
        else:
            priority=operators.find(oper)
            while not A.is_empty() and operators.find(A.last())>priority:
                rpn.append(A.pop())
            A.push(oper)
    while not A.is_empty():
        rpn.append(A.pop())
    return rpn

if __name__== "__main__":
    import doctest
    doctest.testmod()

In [7]:

stack_calculator([1,2,4,"/","+",3,2,-1,"^","*","-"])

0.0

In [8]:
def parse_expression_from_infix_to_rpn(s:list):
    """Dijkstra shunting yard algorithm
    Алгоритм сортировочной станции Дейкстры
    Преобразует математическое выражение из инфиксной(обычной) нотации в
    обратную польскую нотацию
    принимает выражение в виде списка операндов и операций
    возвращает список - выражение в опн 
    Допустимые арифметические операции + - * / ^
    >>> parse_expression_from_infix_to_rpn(['(',3,'+',4,'*','(',2,'-',1,')',')','/',5])
    [3, 4, 2, 1, '-', '*', '+', 5, '/']
    >>> parse_expression_from_infix_to_rpn(['(',2,'-',3,')','*','(',12,'-',10,')','+',4,'/',2])
    [2, 3, '-', 12, 10, '-', '*', 4, 2, '/', '+']
    >>> parse_expression_from_infix_to_rpn([1,"+",2,"/",4,"-",3,'*',2,"^",-1])
    [1, 2, 4, '/', '+', 3, 2, -1, '^', '*', '-']
    """
    operators="-+/*^"
    open_braces="([{"
    close_braces="}])"
    A_list=[[]]
    A=A_list[-1]
    i=0
    rpn=[]
    for oper in s:
        if type(oper)!=str:
            rpn.append(oper)
        elif oper in operators:
            priority=operators.find(oper)
            while A and operators.find(A[-1])>priority:
                rpn.append(A.pop())
            A.append(oper)
        elif oper in open_braces:
            A_list.append([])
            A=A_list[-1]
        elif oper in close_braces:
            while A:
                rpn.append(A.pop())
            A_list.pop()
            A=A_list[-1]
        else:
            rpn.append(oper)
    while A:
        rpn.append(A.pop())
    return rpn

if __name__== "__main__":
    import doctest
    doctest.testmod()

In [9]:
def check_is_string_a_number(number:str):
    """checks is string a number
    return number if it is or False"""
    try:
        res=float(number)
    except ValueError:
        res=False
    else:
        if int(res)==res:
            res=int(res)
        return res
    
def parse_string_to_expression(s:str):
    """ преобразует строку в математическое выражение
    возвращает список эл-тов выражения
    >>> parse_string_to_expression("(3+4*(2-1))/5")
    ['(', 3, '+', 4, '*', '(', 2, '-', 1, ')', ')', '/', 5]
    >>> parse_string_to_expression("(2-3)*(12-10)+4/2")
    ['(', 2, '-', 3, ')', '*', '(', 12, '-', 10, ')', 4, '/', 2]
    >>> parse_string_to_expression("[+-2 +,3]")
    ['[', -1, '*', 2, '+', 0.3, ']']
    """
    expression=[]
    possible_symbols="(){}[]*/^+-"
    prev_symbol_is_number=False
    may_be_power=False
    number=""
    i=0
    for symbol in s:
        if symbol in possible_symbols:
            # write prev number
            if number:
                res=check_is_string_a_number(number)
                if res:
                    expression.append(res)
                else:
                    prev_symbol_is_number=False
                number=""
            #check is uniry operator or pow
            if not prev_symbol_is_number:
                prev_symbol_is_number=False
                if symbol == '-':
                    expression.append(-1)
                    expression.append("*")
                    continue
                elif symbol == '+':
                    continue
                elif symbol=='*'and may_be_power:
                    expression.append("^")
                    continue
            prev_symbol_is_number=False
            if symbol=='*':
                may_be_power=True
                continue
            # add prev *
            if may_be_power:
                expression.append("*")
                may_be_power=False
            expression.append(symbol)
        elif symbol.isnumeric() or symbol in ".,":
            if symbol == ",":
                symbol="."
            number+=symbol
            prev_symbol_is_number=True
            
    if prev_symbol_is_number:
        res=check_is_string_a_number(number)
        if res:
            expression.append(res)
            
    return expression
    

In [10]:
def calculate_expression(s:str):
    """
    Calculates expression from string
    possible operators: -+/*,pow: ^ or **
    braces- (),[],{}
    unary operators: + and -
    numbers: float, int. float can be writen with ',':1,2
    returns result or False if expression can't be calced
    >>> calculate_expression("first number 3 + second number=-3 is ")
    0
    >>> calculate_expression("-[-1**3*(1-3)]/{1+1}")
    -1.0
    >>> calculate_expression("1(+2)")
    Некорректное выражение
    False
    >>> calculate_expression("1*[1-2)")
    Некорректная последовательность скобок в выражении
    False
    """
    if not is_braces_sequense_correct(s):
        print("Некорректная последовательность скобок в выражении")
        return False
    expression=parse_string_to_expression(s)
    expression=parse_expression_from_infix_to_rpn(expression)
    expression=stack_calculator(expression)
    return expression

In [11]:
if __name__== "__main__":
    import doctest
    doctest.testmod()